# Why Dummy data?

In the real dataset a lot of the reservoirs lack inflow data. Hence, resorting to dummy data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import random

In [2]:
# # reservoir A
# A = {}
# A['inflow'] = np.array([100, 120, 130, 90, 60, 30, 20, 30, 20, 90, 150, 100, 80, 70, 50, 40])
# A['storage_change'] = np.array([np.nan, 40, 40, 10, 0, -10, -10, -20, -20])

# Dummy data

In [1]:
from landlab import NetworkModelGrid

In [2]:
y_of_node = (0, 1, 2, 2)
x_of_node = (0, 0, -1, 1)

nodes_at_link=(
    (1, 0),
    (2, 1),
    (3, 1)
)
grid = NetworkModelGrid((y_of_node, x_of_node), nodes_at_link)
grid

<xarray.Dataset>
Dimensions:        (node: 4, link: 3, Two: 2)
Coordinates:
  * node           (node) int64 0 1 2 3
Dimensions without coordinates: link, Two
Data variables:
    mesh           <U1 'a'
    y_of_node      (node) float64 0.0 1.0 2.0 2.0
    x_of_node      (node) float64 0.0 0.0 -1.0 1.0
    nodes_at_link  (link, Two) int64 0 1 2 1 1 3

In [33]:
inflow = grid.add_field(
    'reservoir__unregulated_inflow',
    [3, 2, 1, 1],
    at = 'node',
    units='m3/d',
    clobber=True
)
storage_change = grid.add_field(
    'reservoir__storage_change',
    [0.3, 0.2, -0.1, 0.1],
    at = 'node',
    units='m3/d',
    clobber=True
)
release = grid.add_field(
    'reservoir__release',
    [2.7, 1.8, 1.1, 0.9],
    at = 'node',
    units = 'm3/d',
    clobber=True
)
abstract_elevation = grid.add_field(
    'topographic__elevation',
    [0.0, 1.0, 2.0, 2.0],
    at = 'node',
    units = 'm',
    clobber=True
)
cell_area_at_node = grid.add_field(
    'cell_area_at_node',
    [1, 1, 1, 1],
    at = 'node',
    units = 'm2',
    clobber=True
)


In [34]:
from landlab.components import FlowDirectorSteepest

fdr = FlowDirectorSteepest(grid)
fdr.run_one_step()
fdr

In [23]:
for node in grid.nodes:
    print(grid.adjacent_nodes_at_node[node])

[ 1 -1 -1]
[3 2 0]
[ 1 -1 -1]
[ 1 -1 -1]


In [20]:
grid.links_at_node[0]

array([ 0, -1, -1])

In [105]:
import numpy as np
import numpy.ma as ma

In [115]:
def get_upstream_inflow(grid, node):
    """Calculates the upstream inflow

    Args:
        grid (_type_): _description_
        node (_type_): _description_

    Returns:
        _type_: _description_
    """
    fdr = FlowDirectorSteepest(grid, abstract_elevation)
    fdr.run_one_step()

    number_of_contributors = np.sum(
        fdr.flow_link_incoming_at_node() == 1, axis=1
    )
    downstream_link_id = fdr.link_to_flow_receiving_node
    upstream_contributing_links_at_node = np.where(
        fdr.flow_link_incoming_at_node() == 1, grid.links_at_node, -1
    )

    upstream_links = ma.MaskedArray(upstream_contributing_links_at_node, mask=upstream_contributing_links_at_node==-1)
    upstream_nodes = ma.MaskedArray(fdr.upstream_node_at_link()[upstream_links], mask=upstream_links.mask)
    print(upstream_nodes)
    
    regulated_inflow = ma.sum(
        ma.MaskedArray(grid.at_node['reservoir__release'][upstream_nodes], mask=upstream_links.mask), axis=1
    ).filled(0)

    return regulated_inflow

get_upstream_inflow(grid, 0)

[[1 -- --]
 [3 2 --]
 [-- -- --]
 [-- -- --]]


array([ 1.8,  2. ,  0. ,  0. ])